# Responses API + Code Interpreter: Download a generated file

This notebook mirrors the single-script demo and shows each step.

In [ ]:
# Setup: install (uncomment if needed)
# !pip install --upgrade openai azure-identity

from pathlib import Path
from openai import AzureOpenAI, get_bearer_token_provider  # type: ignore
from azure.identity import DefaultAzureCredential  # type: ignore

ENDPOINT = "https://<resource-name>.openai.azure.com/openai/v1/"
API_VERSION = "preview"
DEPLOYMENT = "gpt-4o"
USE_API_KEY = False  # flip to True to use API key instead of DefaultAzureCredential
AZURE_OPENAI_API_KEY = "<paste-api-key-here-if-you-switch-to-key-mode>"
SCOPE = "https://cognitiveservices.azure.com/.default"
DOWNLOAD_DIR = Path("./downloads")

print("Endpoint:", ENDPOINT)
print("API version:", API_VERSION)
print("Deployment:", DEPLOYMENT)

if USE_API_KEY:
    client = AzureOpenAI(base_url=ENDPOINT, api_key=AZURE_OPENAI_API_KEY, api_version=API_VERSION)
else:
    token_provider = get_bearer_token_provider(DefaultAzureCredential(), SCOPE)
    client = AzureOpenAI(base_url=ENDPOINT, azure_ad_token_provider=token_provider, api_version=API_VERSION)


In [ ]:
# Step 1: Create a response that writes a file via Code Interpreter
instructions = "You are a personal assistant that writes and runs code using the Python tool to answer the question."
response = client.responses.create(
    model=DEPLOYMENT,
    tools=[{"type": "code_interpreter", "container": {"type": "auto"}}],
    instructions=instructions,
    input="Please use the Python tool to create a file called test.txt and write the word 'hello world' to it.",
)
print("Response ID:", getattr(response, "id", None))
if getattr(response, "output_text", None):
    print("Assistant says:", response.output_text[:200])

In [ ]:
# Step 2: Inspect annotations to find files
from typing import Any, Dict, List

def collect_file_annotations(obj: Any) -> List[Dict[str, Any]]:
    results: List[Dict[str, Any]] = []
    def walk(x: Any) -> None:
        if isinstance(x, dict):
            if "file_id" in x:
                results.append({
                    "filename": x.get("filename") or x.get("path") or "unknown",
                    "file_id": x.get("file_id"),
                    "container_id": x.get("container_id"),
                })
            for v in x.values():
                walk(v)
            return
        if isinstance(x, (list, tuple, set)):
            for v in x:
                walk(v)
            return
        for attr in ("annotations", "content", "output", "message", "messages"):
            if hasattr(x, attr):
                try:
                    walk(getattr(x, attr))
                except Exception:
                    pass
        if hasattr(x, "__dict__"):
            try:
                walk(vars(x))
            except Exception:
                pass
    walk(obj)
    seen = set()
    dedup: List[Dict[str, Any]] = []
    for r in results:
        fid = r.get("file_id")
        if fid and fid not in seen:
            seen.add(fid)
            dedup.append(r)
    return dedup

files = collect_file_annotations(response.output)
files

In [ ]:
# Step 3: Download files to ./downloads
DOWNLOAD_DIR.mkdir(exist_ok=True)
for f in files:
    filename = f.get("filename") or "unnamed.bin"
    file_id = f.get("file_id")
    container_id = f.get("container_id")
    if not (file_id and container_id):
        print("Skipping missing metadata:", f)
        continue
    retrieved = client.containers.files.content.retrieve(container_id=container_id, file_id=file_id)
    data = None
    for attr in ("content", "body", "data"):
        if hasattr(retrieved, attr):
            data = getattr(retrieved, attr)
            break
    if data is None and isinstance(retrieved, (bytes, bytearray)):
        data = bytes(retrieved)
    if data is None:
        print("Could not locate bytes for", filename)
        continue
    out_path = DOWNLOAD_DIR / filename
    out_path.write_bytes(data)
    print("Saved", out_path.resolve(), f"({len(data)} bytes)")

In [ ]:
# Step 4: Quick verification (print first line of small text files)
for p in DOWNLOAD_DIR.glob("*"):
    try:
        if p.suffix.lower() in {".txt", ".csv", ".md"} and p.stat().st_size < 64 * 1024:
            print(p.name, "->", p.read_text(errors="replace").splitlines()[0][:120])
        else:
            print(p.name, "->", p.stat().st_size, "bytes")
    except Exception as e:
        print("Could not read", p.name, e)